###### 1.1. Datos:

In [ ]:
import pathlib

#librerías para graficar
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

#tensorfloy
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

In [ ]:
# Estima la eficiencia de combustible de un automóvil
# Usando el dataset Auto MPG, crea el modelo de estimación para la eficiencia de combustible a finales de 1970s y comienzos de 1980s.
# Usa tf.keras API. Para más detalles revisa la guía de Keras.

In [ ]:
#la fuente de los datos
dataset_path = "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data"

In [ ]:
#Cargando los datos en un Pandas DataFrame
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight',
                'Acceleration', 'Model Year', 'Origin']
raw_dataset = pd.read_csv(dataset_path, names=column_names,
                      na_values = "?", comment='\t',
                      sep=" ", skipinitialspace=True)

dataset = raw_dataset.copy()
dataset.tail()

In [ ]:
dataset.info()

###### 1.2. Limpieza de los Datos

In [ ]:
# Nota que algunos datos hacen falta en este dataset

In [ ]:
dataset.isna().sum()  #cuántos datos faltan y en qué columna?

In [ ]:
# Borrando las entradas completa que tienen datos faltantes

In [ ]:
dataset = dataset.dropna()

In [ ]:
dataset.info()

In [ ]:
# Como el atributo "Origin" contiene datos categórigos mas no numéricos, convertimos a one-hot encoding.

In [ ]:
origin = dataset.pop('Origin')  ## método pop() elimina la columna 'Origin' del fataset y lo reescribe. Además guarda la columna eliminada en origin

In [ ]:
origin.unique() ## Categoría de origin

In [ ]:
## "Dummificación" manual: OJO, no limina una de las categorías
dataset['USA']    = (origin == 1)*1.0 ## (origin == 1) es un booleano que devuelve 0 o 1
dataset['Europe'] = (origin == 2)*1.0 ## (origin == 2) es un booleano que devuelve 0 o 1
dataset['Japan']  = (origin == 3)*1.0 ## (origin == 3) es un booleano que devuelve 0 o 1
dataset.tail()

###### 1.3.Divide los datos en conjunto de entrenamiento y conjunto de evaluación

In [1]:
## Otra forma diferente a "train_test_split" de sklearn para dividir los datos en entrenamiento y prueba
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [ ]:
## Matriz de diagramas de dispersión entre las columnas originalmente numéricas
sns.pairplot(train_dataset[["MPG", "Cylinders", "Displacement", "Weight"]], diag_kind="kde", kind= "reg")

###### 1.5.Verifica las estadísticas descriptivas

In [ ]:
train_stats = train_dataset.describe() ## Guarda el dataset con las estadísticas descriptivas
train_stats.pop("MPG")                 ## Elimina la columna correspondiente a la variable respuesta
train_stats = train_stats.transpose()  ## Transpone el fataframe
train_stats ## Nótese la diferencia de escalas en las variables

###### 1.6.Separa el valor objetivo en atributo

In [ ]:
train_labels = train_dataset.pop('MPG') ## Etiquetas para los datos de entrenamiento
test_labels = test_dataset.pop('MPG')   ## Etiquetas para los datos de prueba

###### 1.7.Normaliza los Datos

Este enfoque `NO` es recomendado porque no guarda la información de normalización

In [ ]:
def norm(x):
    return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [ ]:
## observe todas las medias cercanas a 0 y desviacines estándar a 1
normed_test_data.describe().loc[["mean", "std"],]

###### 1.8.Crea el modelo

In [ ]:
# Construye el modelo secuencial con una capa densamente conectada.
# La capa de salida devuelve los valores consecutivos.

In [ ]:
def build_model(): ## Función auxiliar para construir y compilar la red
    model = keras.Sequential(#---------------------------------------------- Sequential permite construir la aequitectura por capas
        [                    #---------------------------------------------- layers permite especificar la información por capa. Dense significa totalmente conectada.
              layers.Dense(  64 #------------------------------------------- Número de nodos (neuronas) de la capa
                           , activation='relu' #---------------------------- función de activación de la capa
                           , input_shape=[len(train_dataset.keys())] #------ Número de variables de la capa de entrada
                             )
            , layers.Dense(64, activation='relu') #------------------------- Segunda capa interna, también con RELU y 64 neuronas
            , layers.Dense(1)]) #------------------------------------------- capa de salida, una neurona y función de activación LINEAR (por defecto)
    optimizer = tf.keras.optimizers.RMSprop(0.001) #------------------------ selección del algoritmo de optimización y tasa de aprendizaje (0.001)
    model.compile( #-------------------------------------------------------- El método compile() Configura el modelo para entrenamiento.
                  loss='mse', #--------------------------------------------- Elige la funcipon de pérdida  o costo (mse: Mean Square Error).
                  optimizer=optimizer, #------------------------------------ Elige el algoritmo de optimización
                  metrics=['mae', 'mse', 'mape']) #------------------------- Selecciona las métricas de desempeño
    return model

model = build_model()

###### 1.9.Verifica el modelo

In [ ]:
print(len(train_dataset.keys())) ## Número de variables de entrada: 9
model.summary()
# [ 9pesos + 1sesgo]x[64 neuronas] = 640
# [64pesos + 1sesgo]x[64 neuronas] = 4160
# [64pesos + 1sesgo]x[1  neurona]  = 65

In [ ]:
# Llama el método model.predict colocando 10 muestras del conjunto de entrenamiento en un lote.
# IMPORTANTE: En este punto, el modelo NO está entrenado, pero los pesos se han inicializado y es posible hacer predicciones, generalmente MALAS

In [ ]:
example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
example_result

In [ ]:
## PESOS INICIALES
model.get_weights()#[0][0].shape

###### 1.10.Entrena el modelo

In [ ]:
%%time
# Coloca un punto (.) para indicar el proceso de entrenamiento al final de cada época (epoch)
# Se crea la clase PrintDot, la cual hereda de la clase keras.callbacks.Callback
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: print('') #-------------- imprime " . " por cada época y las agrupa en lotes de 100
        print('.', end='')

EPOCHS = 1000 #--------------------------------------- define el número de épocas

history = model.fit( #-------------------------------- Inicia el entrenamineto y guarda el historial
                    normed_train_data, #-------------- X_train
                    train_labels, #------------------- Y_train
                    epochs=EPOCHS,#------------------- épocas
                    validation_split = 0.2, #--------- Proporción de datos para proceso de validación
                    verbose=0, #---------------------- Silencia las notificaciones de progreso automático
                    callbacks=[PrintDot()] #---------- Invoca la clase PrintDot para darle formato a las notificaciones de progreso automático
                    )
## Desempeño: Wall time: 1min 22s (CPU)

In [ ]:
print(len("...................................................................................................."))
type(history)

In [ ]:
# visualiza el rendimiento del proceso de entrenamiento usando las estadísticas en
# el objeto "history".

In [ ]:
hist = pd.DataFrame(history.history) #--- Crea un dataframe con la historia de entrenamiento
hist['epoch'] = history.epoch        #---   Agrega una columna adicional con el índice de cafa época
hist.tail()

In [ ]:
hist.head()

Graficando el error

In [ ]:
import matplotlib.pyplot as plt

def plot_history(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    plt.figure(figsize=(15,8))

    plt.subplot(1,3,1)
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error [MPG]')
    plt.plot(hist['epoch'], hist['mae'],label='Error de Entrenamiento')
    plt.plot(hist['epoch'], hist['val_mae'],label = 'Error de Validación')
    plt.ylim([0,5])
    plt.legend()

    plt.subplot(1,3,2)
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error [$MPG^2$]')
    plt.plot(hist['epoch'], hist['mse'],label='Error de entrenamiento')
    plt.plot(hist['epoch'], hist['val_mse'],label = 'Error de Validación')
    plt.ylim([0,20])
    plt.legend()

    plt.subplot(1,3,3)
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Percentage Error [$MAPE$]')
    plt.plot(hist['epoch'], hist['mape'],label='Error de entrenamiento')
    plt.plot(hist['epoch'], hist['val_mape'],label = 'Error de Validación')
    plt.ylim([0,20])
    plt.legend()
    plt.show()

plot_history(history)

In [ ]:
# Al modificar el método model.fit, el entrenamiento se detiene automáticamente cuando la variación no mejora.
# Utiliza EarlyStopping callback para verificar el proceso de entrenamiento en cada época.

# Detener automáticamente el entrenamiento cuando no haya más mejoras con respecto al número
# especificado de épocas.

In [ ]:
%%time
model = build_model()

# El parámetro 'patience' es la cantidad de épocas que se usará para verificar la mejora.
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(normed_train_data, train_labels, epochs=EPOCHS,
                    validation_split = 0.2, verbose=0, callbacks=[early_stop, PrintDot()])

plot_history(history)

In [ ]:
len(history.epoch) ## se detuvo en la época 70.

###### 1.11.Verificar el rendimiento del modelo en el conjunto de entrenamiento.

In [ ]:
loss, mae, mse, mape = model.evaluate(normed_test_data, test_labels, verbose=2)

print("Mean absolute error del conjunto de evaluación: {:5.2f} MPG".format(mae))

In [ ]:
# model.evaluate(normed_test_data, test_labels, verbose=2)
print(round(mape,2), "%", sep="")

###### 1.12. Predicción

In [3]:
#utilizamos el método ".predict" para utilizar el modelo para hacer predicción
test_predictions = model.predict(normed_test_data).flatten()

plt.scatter(test_labels, test_predictions)
plt.xlabel('Valores Reales [MPG]')
plt.ylabel('Predicciones [MPG]')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 100], [-100, 100]) ## para agregar la línea recta "y = x"

In [ ]:
# model.predict(normed_test_data).flatten().shape#
import numpy as np

np.corrcoef(test_labels, test_predictions)[1,0] ## Alta correlación lineal entre lo obsevado y lo predicho es un buen indicador del modelo